# Chapter 47: Model Compression - Quantization and Pruning
Reduce model size while preserving accuracy.


In [ ]:
import numpy as np
print("Model Compression ready")


In [ ]:
np.random.seed(42)
W=np.random.randn(256,256).astype("float32")
mn,mx=W.min(),W.max(); scale=(mx-mn)/255; zp=-round(mn/scale)
q=np.clip(np.round(W/scale+zp),0,255).astype("uint8")
dq=(q.astype("float32")-zp)*scale; mse=float(np.mean((W-dq)**2))
print(f"FP32: {W.nbytes:,}B | INT8: {q.nbytes:,}B | ratio: {W.nbytes/q.nbytes:.1f}x | MSE: {mse:.6f}")


In [ ]:
for sp in [0.3,0.5,0.7,0.9]:
    thr=np.percentile(np.abs(W),sp*100)
    mask=np.abs(W)>thr; actual=1-mask.sum()/mask.size
    print(f"Target {sp:.0%}: actual={actual:.1%}, nonzero={mask.sum():,}/{W.size:,}")
print("Compression analysis for 7B params:")
orig=7_000_000_000
for bits in [16,8,4,2]: print(f"  {bits}-bit: {32/bits:.1f}x compression, ~{orig*bits/8/1e9:.1f}GB")
for sp in [0.5,0.7,0.9]: print(f"  {sp:.0%} prune: ~{orig*4*(1-sp)/1e9:.1f}GB")
